Imports

In [ ]:
import os, sys
import django
from datetime import datetime
sys.path.append('../') # add path to project root dir

os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_appsh

django.setup()

Declare PIDs

In [ ]:
mother_pid = "B142-040990116-1"
child_c_pid = "B142-040990116-1-10"
child_b_pid = "B142-040990116-1-60"

Clone CRF's and assign them to a different schedule

In [ ]:
from flourish_caregiver.models import MaternalVisit
from edc_appointment.models import Appointment
from edc_appointment.constants import NEW_APPT, IN_PROGRESS_APPT

visit_list = ['2001M','2002M', '2003M', '2004M', '2005M','2006M']

for visit in visit_list:
    # clone maternal visit and associate with appointment from cohort c


    # find appointment on cohort c
    appointment_b = Appointment.objects.filter(subject_identifier=mother_pid,visit_code=visit, schedule_name='b_quarterly1_schedule1').first()

    appointment_c = Appointment.objects.filter(subject_identifier=mother_pid,visit_code=visit, schedule_name='c_quarterly1_schedule1').first()

    maternal_visit_b = getattr(appointment_b, 'maternalvisit')

    # clone maternal visit for cohort c
    maternal_visit_b.id = None
    maternal_visit_b.schedule_name = appointment_c.schedule_name
    maternal_visit_b.visit_schedule_name = appointment_c.visit_schedule_name
    maternal_visit_b.appointment = appointment_c
    maternal_visit_b.save()

    # update appointment
    appointment_c.appt_status = NEW_APPT
    appointment_c.save()

put other child on follow-up schedule

In [ ]:
from flourish_caregiver.helper_classes.fu_onschedule_helper import FollowUpEnrolmentHelper


class CustomEnrolmentHelper(FollowUpEnrolmentHelper):

        def get_latest_completed_appointment(self, subject_identifier, cohort, schedule_number):

            appts = Appointment.objects.filter(subject_identifier = mother_pid, visit_code='2002M' ,schedule_name='c_quarterly1_schedule1')

            if appts:
                latest = appts.order_by('timepoint').last()
                return latest


schedule_enrol_helper = CustomEnrolmentHelper(
            subject_identifier=mother_pid,
            cohort='c')

# put child on follow-up schedule

schedule_enrol_helper.activate_fu_schedule()

Move crf on follow-up schedule from child on cohort b to cohort c


In [ ]:
from edc_metadata.models import CrfMetadata
from flourish_caregiver.models import MaternalVisit
# create subjectschedule history again

b_fu1_schedule1_visits = ['3000M']

b_fu1_schedule1_visit_crf = []
for visit in b_fu1_schedule1_visits:

    # find all the crfs on a specific visit at schedule b_quarterly1_schedule1
    crf_list = [i.model for i in CrfMetadata.objects.filter(subject_identifier=mother_pid,
                                                            visit_code=visit,schedule_name='b_fu1_schedule1')]
    # create a dict of visit with a list of existing crf objects
    b_fu1_schedule1_visit_crf.append({visit : crf_list})

list(b_fu1_schedule1_visit_crf[0].values())

Match date on onshedule model

In [ ]:
from flourish_caregiver.models import OnScheduleCohortCFU
from flourish_caregiver.models import OnScheduleCohortBFU


onsch_c = OnScheduleCohortCFU.objects.filter(
    subject_identifier=mother_pid, schedule_name='c_fu1_schedule1'
).first()


onsch_b = OnScheduleCohortBFU.objects.filter(
    subject_identifier=mother_pid, schedule_name='b_fu1_schedule1'
).first()

onsch_c.onschedule_datetime = onsch_b.onschedule_datetime
onsch_c.save()




from edc_visit_schedule.models import SubjectScheduleHistory

subj_sch_history_c = SubjectScheduleHistory.objects.filter(subject_identifier=mother_pid,schedule_name='c_fu1_schedule1').first()

subj_sch_history_b = SubjectScheduleHistory.objects.filter(subject_identifier=mother_pid,schedule_name='b_fu1_schedule1').first()

subj_sch_history_c.onschedule_datetime = subj_sch_history_b.onschedule_datetime
subj_sch_history_c.save()

Match date on offschedule model for object created when enrolling on follow up schedule

In [ ]:
from flourish_caregiver.models import CaregiverOffSchedule
# find Quarterly Schedule offschedule
offsch_quart_c = CaregiverOffSchedule.objects.filter(
    subject_identifier=mother_pid, schedule_name='c_quarterly1_schedule1'
).first()

offsch_quart_b = CaregiverOffSchedule.objects.filter(
    subject_identifier=mother_pid, schedule_name='b_quarterly1_schedule1'
).first()

offsch_quart_c.offschedule_datetime = offsch_quart_b.offschedule_datetime
offsch_quart_c.save()

Transfer crf on follow up cohor c

In [ ]:
visit_code = '3000M'
print("Processing", visit_code)

matenal_visit_b = MaternalVisit.objects.filter(subject_identifier=mother_pid,visit_code=visit_code, schedule_name='b_fu1_schedule1').first()

# find appointment on cohort c since it already exists
appointment_c = Appointment.objects.filter(subject_identifier=mother_pid,visit_code=visit_code, schedule_name='c_fu1_schedule1').first()

# corresponding appointment
appointment_b = Appointment.objects.filter(subject_identifier=mother_pid,visit_code=visit_code, schedule_name='b_fu1_schedule1').first()

appointment_c.appt_type = appointment_b.appt_type
appointment_c.appt_datetime = appointment_b.appt_datetime
appointment_c.appt_status = appointment_b.appt_status
appointment_c.appt_reason = appointment_b.appt_reason
appointment_c.save()

#clone
matenal_visit_b.id = None
matenal_visit_b.schedule_name = 'c_fu1_schedule1'
matenal_visit_b.visit_schedule_name = 'c1_visit_schedule1'
matenal_visit_b.appointment = appointment_c
matenal_visit_b.save()


crf_list_fu = list(b_fu1_schedule1_visit_crf[0].values())


transfer crfs on followup visit to other child

In [ ]:

for model in crf_list_fu[0]:
    if model:
        try:
            crf = django_apps.get_model(model).objects.get(maternal_visit__subject_identifier=mother_pid, maternal_visit__visit_code=visit_code, maternal_visit__schedule_name='b_fu1_schedule1')

        except django_apps.get_model(model).DoesNotExist:
            print(model,"crf not found")
        else:
            print("transferring",model)
            crf.id = None
            crf.maternal_visit = matenal_visit_b
            crf.save()

            # delete old crf
            old_crf = django_apps.get_model(model).objects.get(maternal_visit__subject_identifier=mother_pid,maternal_visit__visit_code=visit_code, maternal_visit__schedule_name='b_fu1_schedule1')
            old_crf.delete()

In [ ]:

# delete old maternal visit
maternal_vist_b = MaternalVisit.objects.filter(subject_identifier='B142-040990116-1',visit_code=visit_code, schedule_name='b_fu1_schedule1').first()

if maternal_vist_b:
    for i in maternal_vist_b.metadata['crf']:
        if i.entry_status == 'KEYED':
            i.entry_status=''
            i.save()
    maternal_vist_b.metadata_delete_for_visit()
    appointment = maternal_vist_b.appointment
    maternal_vist_b.delete()

    print(appointment.maternalvisit)
    appointment.maternalvisit = None
    appointment.appt_status = NEW_APPT
    appointment.save()

delete followup onschedule , offshcedule, subjectschedulehistory for cohort b

In [ ]:


from flourish_caregiver.models import CaregiverOffSchedule,OnScheduleCohortBFU,OnScheduleCohortBFUQuarterly, OnScheduleCohortBQuarterly
from edc_visit_schedule.models import SubjectScheduleHistory

# get quarterly onsched
try:
    onschedule_object = OnScheduleCohortBQuarterly.objects.get(subject_identifier=mother_pid,child_subject_identifier=child_b_pid)
except:
    pass
else:
    print(onschedule_object)

# delete fu onschedule
try:
    onschedule_object = OnScheduleCohortBFU.objects.get(subject_identifier=mother_pid,child_subject_identifier=child_b_pid)
except:
    pass
else:
    print("onschedule_object delete")
    onschedule_object.delete()


try:
    onschedule_object_fu = OnScheduleCohortBFUQuarterly.objects.get(subject_identifier=mother_pid,child_subject_identifier=child_b_pid)
except:
    pass
else:

    print("onschedule_object_fu delete")
    onschedule_object_fu.delete()



# put cohor b child back on b_quarterly1_schedule1 schedule
try:
    subj_sch_history_b = SubjectScheduleHistory.objects.get(
        subject_identifier=mother_pid,
        schedule_name =  'b_quarterly1_schedule1',
        schedule_status = 'offschedule'
    )

except:
    pass
else:
    subj_sch_history_b.offschedule_datetime=None
    subj_sch_history_b.schedule_status='onschedule'
    subj_sch_history_b.save()
